# PENGENALAN EMOSI

## penjelasan data

Toronto Emotional Speech Set (TESS) adalah kumpulan data yang terdiri dari rekaman suara yang diucapkan oleh seorang pria dan wanita. Ada 1400 file audio dan 200 kata per individu. File audio berformat .wav. Setiap frase diucapkan dalam salah satu dari tujuh suasana hati emosional yang berbeda. Setiap file memiliki sekitar 2 detik audio yang direkam di dalamnya.

Sebuah set dari 200 kata target diucapkan dalam frase pembawa "Say the word _____" oleh dua aktris (berusia 26 dan 64 tahun) dan rekaman dibuat dari set yang menggambarkan masing-masing tujuh emosi (kemarahan, jijik, takut, kebahagiaan, kejutan yang menyenangkan, kesedihan, dan netral)

Model yang dikembangkan menggunakan dataset TESS dapat mengenali emosi seperti tenang, bahagia, takut, jijik, marah, netral, terkejut dan sedih. Dataset ini digunakan untuk pengenalan emosi dalam pidato.


## input data dan import semua library yang di butuhkan

In [1]:
import os

# Ganti 'nama_folder' dengan nama folder yang ingin Anda masuki
path = 'suara'

# Gunakan metode `chdir` untuk berpindah ke folder tersebut
os.chdir(path)

# Sekarang Anda berada di dalam folder tersebut

In [2]:
folders=['YAF_sad','YAF_pleasant_surprised','YAF_neutral',
         'YAF_happy','YAF_fear','YAF_disgust','YAF_angry',
         'OAF_Sad','OAF_Pleasant_surprise','OAF_neutral',
         'OAF_happy','OAF_Fear','OAF_disgust',
         'OAF_angry',
         ]

In [3]:
# Import Library
import os
import librosa
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, mode
import pickle
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## PREPROCESSING

### membuat fungsi untuk ekstraksi fitur

$$
ZCR = \frac{1}{T} \sum_{n=1}^{T-1} \left| \text{sign}(x[n]) - \text{sign}(x[n-1]) \right|
$$


di sini saya hanya menggunakan 1 ektraksi fitur yaitu zero crossing rate

In [4]:
def calculate_statistics(audio_path):
    y, sr = librosa.load(audio_path)

    # UNTUK MENGHITUNG NILAI STATISTIKA
    mean = np.mean(y)
    std_dev = np.std(y)
    max_value = np.max(y)
    min_value = np.min(y)
    median = np.median(y)
    skewness = skew(y)  # Calculate skewness
    kurt = kurtosis(y)  # Calculate kurtosis
    q1 = np.percentile(y, 25)
    q3 = np.percentile(y, 75)
    mode_value, _ = mode(y)  # Calculate mode
    iqr = q3 - q1

    # UNTUK MENGHITUNG NILAI ZCR
    zcr_mean = np.mean(librosa.feature.zero_crossing_rate(y=y))
    zcr_median = np.median(librosa.feature.zero_crossing_rate(y=y))
    zcr_std_dev = np.std(librosa.feature.zero_crossing_rate(y=y))
    zcr_kurtosis = kurtosis(librosa.feature.zero_crossing_rate(y=y)[0])
    zcr_skew = skew(librosa.feature.zero_crossing_rate(y=y)[0])



    return [zcr_mean, zcr_median, zcr_std_dev, zcr_kurtosis, zcr_skew]

### memberi label pada suara dan menajdikannya dataframe agar bisa di convert ke CSV

pertama saya membuat list untuk menyimpan bernama features lalu saya memberi label dengan nama folder dan menjadikanya csv
dan 5 fitur = ZCR Mean,ZCR Median, ZCR Std Dev, ZCR Kurtosis, ZCR Skew

In [5]:
features =[]

In [6]:
for folder in folders:
    folder_path = f'{folder}'
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            audio_path = os.path.join(folder_path, filename)
            statistics = calculate_statistics(audio_path)
            features.append([folder, filename] + statistics)

KeyboardInterrupt: 

In [34]:
# Membuat DataFrame dari data
columns =  ['Label', 'File'] + ['ZCR Mean', 'ZCR Median', 'ZCR Std Dev', 'ZCR Kurtosis', 'ZCR Skew']
df = pd.DataFrame(features, columns=columns)

In [35]:
# Menampilkan file CSV
df

,Label,File,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew
0,YAF_sad,YAF_back_sad.wav,0.128581,0.033203,0.225249,3.231511,2.213080
1,YAF_sad,YAF_bar_sad.wav,0.111230,0.036133,0.196220,5.189765,2.569729
2,YAF_sad,YAF_base_sad.wav,0.196274,0.042480,0.259866,-0.207301,1.226778
3,YAF_sad,YAF_bath_sad.wav,0.145458,0.039062,0.222568,2.644984,1.998682
4,YAF_sad,YAF_bean_sad.wav,0.105494,0.035156,0.194269,5.715395,2.682275
...,...,...,...,...,...,...,...
2795,OAF_angry,OAF_witch_angry.wav,0.095215,0.070801,0.084418,2.776079,1.733152
2796,OAF_angry,OAF_yearn_angry.wav,0.083991,0.051270,0.091742,7.208814,2.744011
2797,OAF_angry,OAF_yes_angry.wav,0.115001,0.060059,0.118703,3.370580,2.012270
2798,OAF_angry,OAF_young_angry.wav,0.085005,0.059082,0.090037,5.366187,2.422199


In [9]:
df.to_csv('emosi_5_fitur.csv',index=False)

### standarisasi data

saya ingin melakukan standarisasi data menggunakan standard scaller dan menjadikan nya csv lagi untuk bisa di proses lebih lanjut $$
z = \frac{{x - \mu}}{{\sigma}}
$$


In [50]:
from sklearn.preprocessing import StandardScaler
# Baca file CSV
dn = pd.read_csv("emosi_5_fitur.csv")
# Tentukan kolom yang akan distandarisasi
kolom = ['ZCR Mean', 'ZCR Median', 'ZCR Std Dev', 'ZCR Kurtosis', 'ZCR Skew']
# Inisialisasi StandardScaler
scaler = StandardScaler()
# Lakukan standarisasi pada kolom yang telah ditentukan
dn[kolom] = scaler.fit_transform(dn[kolom])
# Simpan DataFrame yang telah distandarisasi ke dalam file CSV baru
dn.to_csv("emosi2normalisasi.csv", index=False)


In [51]:
norm=pd.read_csv('emosi2normalisasi.csv')
norm

,Label,File,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew
0,YAF_sad,YAF_back_sad.wav,-0.321317,-1.052521,0.940845,0.204442,0.495211
1,YAF_sad,YAF_bar_sad.wav,-0.643634,-0.962423,0.324040,1.185171,1.219225
2,YAF_sad,YAF_base_sad.wav,0.936230,-0.767212,1.676354,-1.517779,-1.507027
3,YAF_sad,YAF_bath_sad.wav,-0.007778,-0.872326,0.883874,-0.089302,0.059975
4,YAF_sad,YAF_bean_sad.wav,-0.750198,-0.992456,0.282586,1.448416,1.447698
...,...,...,...,...,...,...,...
2795,OAF_angry,OAF_witch_angry.wav,-0.941158,0.103730,-2.051468,-0.023647,-0.479064
2796,OAF_angry,OAF_yearn_angry.wav,-1.149659,-0.496920,-1.895834,2.196348,1.573026
2797,OAF_angry,OAF_yes_angry.wav,-0.573586,-0.226627,-1.322985,0.274090,0.087558
2798,OAF_angry,OAF_young_angry.wav,-1.130833,-0.256660,-1.932072,1.273526,0.919734


### memisahkan fitur dan label untuk di jadikan data train dan test

saya memisahkan label dan fitur agar bisa di split data train dan data test 20% data train 80%

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pickle import dump

In [53]:
# Baca data dari file CSV
dataknn= pd.read_csv('emosi2normalisasi.csv')
# Pisahkan fitur (X) dan label (y)
X = dataknn.drop(['Label','File'], axis=1)  # Ganti 'target_column' dengan nama kolom target
y = dataknn['Label']
# split data into train and test sets
print(X)


      ZCR Mean  ZCR Median  ZCR Std Dev  ZCR Kurtosis  ZCR Skew
0    -0.321317   -1.052521     0.940845      0.204442  0.495211
1    -0.643634   -0.962423     0.324040      1.185171  1.219225
2     0.936230   -0.767212     1.676354     -1.517779 -1.507027
3    -0.007778   -0.872326     0.883874     -0.089302  0.059975
4    -0.750198   -0.992456     0.282586      1.448416  1.447698
...        ...         ...          ...           ...       ...
2795 -0.941158    0.103730    -2.051468     -0.023647 -0.479064
2796 -1.149659   -0.496920    -1.895834      2.196348  1.573026
2797 -0.573586   -0.226627    -1.322985      0.274090  0.087558
2798 -1.130833   -0.256660    -1.932072      1.273526  0.919734
2799 -1.324499   -0.587017    -1.750587      2.491151  1.906111

[2800 rows x 5 columns]


In [54]:
X_train,X_test,y_train, y_test= train_test_split(X, y, random_state=1, test_size=0.2)
# define scaler


## modeling KNN

1. **Menghitung Jarak:**
   Untuk setiap titik data $X_i$, kita menghitung jaraknya ke semua titik data lainnya, biasanya menggunakan Euclidean distance atau metrik jarak lainnya.
   
   $$
   d(X_i, X_j) = \sqrt{\sum_{k=1}^{n}(X_{i,k} - X_{j,k})^2}
   $$

2. **Menentukan Tetangga Terdekat:**
   Kita memilih $k$ titik terdekat dengan jarak terkecil dari $X_i$.

3. **Voting atau Rata-rata:**
   Dalam klasifikasi, kita melakukan voting mayoritas untuk menentukan label kelas dari $X_i$ berdasarkan label kelas tetangga terdekat. Dalam regresi, kita mungkin mengambil rata-rata nilai dari tetangga terdekat sebagai prediksi.


In [55]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

lalu saya mencari k terbaik dari 1-30

In [56]:
best_accuracy = 0
best_k = 0

# Perulangan untuk mencari akurasi tertinggi
for n in range(1, 30):  # Ganti dengan rentang nilai k yang ingin Anda uji
    knn = KNeighborsClassifier(n_neighbors=n, metric='euclidean')
    knn.fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)

    print(f'Akurasi dengan k={n} adalah {accuracy}')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = n

print(f'Akurasi terbaik adalah {best_accuracy} dengan nilai k={best_k}')

Akurasi dengan k=1 adalah 0.45
Akurasi dengan k=2 adalah 0.4392857142857143
Akurasi dengan k=3 adalah 0.4607142857142857
Akurasi dengan k=4 adalah 0.49642857142857144
Akurasi dengan k=5 adalah 0.48214285714285715
Akurasi dengan k=6 adalah 0.49642857142857144
Akurasi dengan k=7 adalah 0.4857142857142857
Akurasi dengan k=8 adalah 0.48214285714285715
Akurasi dengan k=9 adalah 0.4857142857142857
Akurasi dengan k=10 adalah 0.48214285714285715
Akurasi dengan k=11 adalah 0.48392857142857143
Akurasi dengan k=12 adalah 0.475
Akurasi dengan k=13 adalah 0.475
Akurasi dengan k=14 adalah 0.4875
Akurasi dengan k=15 adalah 0.4928571428571429
Akurasi dengan k=16 adalah 0.5125
Akurasi dengan k=17 adalah 0.5
Akurasi dengan k=18 adalah 0.5053571428571428
Akurasi dengan k=19 adalah 0.49464285714285716
Akurasi dengan k=20 adalah 0.4928571428571429
Akurasi dengan k=21 adalah 0.49464285714285716
Akurasi dengan k=22 adalah 0.49464285714285716
Akurasi dengan k=23 adalah 0.5035714285714286
Akurasi dengan k=24 a

In [57]:
accuracy = accuracy_score(y_test,y_pred)
print("Akurasi:",accuracy)

Akurasi: 0.5035714285714286
